In [1]:
import torch
import torch.nn as nn
from torch import optim

import pandas as pd
import numpy as np
from tqdm import tqdm
import importlib
import pickle

import models
import DataLoader
import utils
import main

# test

In [2]:
# features
masts = [0., 1., 2., 3.]
educd = [0., 1., 2., 3., 4., 5., 6.]
trdtp = [0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29.]
poscd = [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 99.]
gender_code = [0., 1.]
age = [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]
labels = [0, 2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]

# test loader
importlib.reload(DataLoader)
path = './dataset/user_features4/'
idxs = torch.arange(0, 500000)
data_loader = DataLoader.dataLoader(path, idxs, batch_size=10000, train=False, shuffle=False)
# data_loader = DataLoader.DataSet(path, train_idxs)

In [3]:
importlib.reload(models)
savePath ='./models/checkpoints_hoo2.pt'
checkpoint = torch.load(savePath, map_location=torch.device('cpu'))

featureType = checkpoint['featureType']
modelType = checkpoint['modelType']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

LSTM = models.LSTMModel(featureType, modelType, device=device).to(device)
LSTM.load_state_dict(checkpoint['model'])
LSTM.train(False)
device

device(type='cuda')

In [4]:
result = []
idx = []
LSTM.train(False)
for inputs in data_loader:
    inputs = inputs.to(device)
    with torch.no_grad():
        idx.append(inputs[:, 0:1, 0])
        
        inputs = inputs[:, :, 1:-3]
        outputs = LSTM(inputs)[:, -1, :]
        result.append(outputs)

result = torch.cat(result, dim=0).cpu().numpy()
idx = torch.cat(idx, dim=0).cpu().numpy()

In [5]:
d = {'result': result, 'idx': idx}
with open('./result/result6.pickle', 'wb') as f:
    pickle.dump(d, f)

# get csv

In [6]:
with open('./result/result6.pickle', 'rb') as f:
    res = pickle.load(f)

In [7]:
result, idx = res['result'], res['idx']

In [8]:
r = result[:, 1:].argsort(axis=1)[:, ::-1] + 1
r = r[:, :3]

In [9]:
idx = idx.astype('int')

In [10]:
# trans
labels = np.array([0, 2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48])
def trans(r, labels):
    return labels[r]

In [12]:
rs = trans(r, labels)

In [13]:
outputs = np.concatenate([idx, rs], axis=1)

In [14]:
outputs[:, 1]

array([ 6, 15, 37, ..., 37, 37,  2])

In [15]:
column_names = ["chid", "top1", "top2", "top3"]
outputs = pd.DataFrame(outputs, columns=column_names)
outputs

,chid,top1,top2,top3
0,10128239,6,37,36
1,10077943,15,36,2
2,10277876,37,10,15
3,10364842,37,6,10
4,10392717,48,15,2
...,...,...,...,...
499995,10433239,10,15,2
499996,10235119,15,2,36
499997,10267595,37,10,15
499998,10160606,37,6,10


In [16]:
outputs.to_csv('./result/result6.csv', index=False)

# 統計類別出現次數

In [2]:
importlib.reload(DataLoader)
# features = ['chid', 'masts', 'educd', 'trdtp', 'poscd', 'gender_code',
#                'age', 'label1', 'label2', 'label3', 'tx1', 'tx2', 'tx3']
path = './dataset/user_features2/'
train_idxs = torch.arange(0, 500000)
data_loader = DataLoader.dataLoader(path, train_idxs, batch_size=10000, train=False)

r = []
for i in data_loader:
    r.append(i)
r = torch.cat(r, dim=0)

In [11]:
re = r.reshape(-1, 13)[:, -6: -3].type(torch.int64)

In [28]:
weight = []
for i in range(17):
    c = (re == i).sum()
    weight.append(c)
weight = torch.tensor(weight)
# weight = weight / weight.sum()

In [29]:
weight

tensor([19134461,  1576335,   498543,  1847943,   841152,   375463,  1918747,
          432460,   823159,   324203,   327686,   443574,   340725,  1769071,
         3495142,   456382,  1394954])

In [36]:
weights = weight / weight.sum()
weights = 1 / (weights)

In [37]:
(weights / weights.sum())

tensor([0.0020, 0.0242, 0.0765, 0.0206, 0.0453, 0.1016, 0.0199, 0.0882, 0.0463,
        0.1176, 0.1164, 0.0860, 0.1119, 0.0216, 0.0109, 0.0836, 0.0273])

In [41]:
c = pd.read_csv('dataset/data_information/id_list.csv', index_col=None)
c

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
...,...,...,...,...
499995,10433239,18,48,25
499996,10235119,6,36,12
499997,10267595,21,26,15
499998,10160606,39,15,13


In [44]:
c['chid'] == 10128239

0          True
1         False
2         False
3         False
4         False
          ...  
499995    False
499996    False
499997    False
499998    False
499999    False
Name: chid, Length: 500000, dtype: bool